In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

questions = {}
pairs_train = []
y_train = []
with open('data/train.csv','r',encoding='utf8') as f:
    for line in f:
        l = line.split(',')
        if l[1] not in questions:
            questions[l[1]] = l[3]
        if l[2] not in questions:
            questions[l[2]] = l[4]

        pairs_train.append([l[1], l[2]])
        y_train.append(int(l[5][:-1]))

pairs_test = []
with open('data/test.csv','r',encoding='utf8') as f:
    for line in f:
        l = line.split(',')
        if l[1] not in questions:
            questions[l[1]] = l[3]
        if l[2] not in questions:
            questions[l[2]] = l[4][:-1]

        pairs_test.append([l[1], l[2]])


ids2ind = {}
for qid in questions:
    ids2ind[qid] = len(ids2ind) 

vec = TfidfVectorizer()
A = vec.fit_transform(questions.values())

N_train = len(pairs_train)
X_train = np.zeros((N_train, 3))
for i in range(len(pairs_train)):
    q1 = pairs_train[i][0]
    q2 = pairs_train[i][1]
    X_train[i,0] = cosine_similarity(A[ids2ind[q1],:], A[ids2ind[q2],:])
    X_train[i,1] = len(questions[q1].split()) + len(questions[q2].split())
    X_train[i,2] = abs(len(questions[q1].split()) - len(questions[q2].split()))


N_test = len(pairs_test)
X_test = np.zeros((N_test, 3))
for i in range(len(pairs_test)):
    q1 = pairs_test[i][0]
    q2 = pairs_test[i][1]
    X_test[i,0] = cosine_similarity(A[ids2ind[q1],:], A[ids2ind[q2],:])
    X_test[i,1] = len(questions[q1].split()) + len(questions[q2].split())
    X_test[i,2] = abs(len(questions[q1].split()) - len(questions[q2].split()))

clf = RandomForestClassifier(n_estimators=500, max_depth=3, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)

with open('submissions/submission_file.csv', 'w') as f:
    f.write('Id,Score\n')
    for i in range(y_pred.shape[0]):
        f.write(str(i)+','+str(y_pred[i][1])+'\n')


C:\Users\Mohamed\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
